## Deeplearning network 
### individuare la legge di correlazione fra due dati

il presente si propone l' esercizio di :

1. creare il dataset in excel di

    500 triple
    
    --> x1 e x2 e y tali che EQ.A= " 3*x1  = y " 
    
     --> x1 e x2 e y tali che EQ.B=" 5*x1  = y "
    
    
2. creare il dataset in excel di z che sia 0 se EQ.A e 1 se EQ.B


3. esportare il dataset sottoforma di files .CSV
    train.CSV
    test.CSV
    validate.CSV


4. dividere il dataset in "train_data" e "train_solution" ,
                           "test_data" e "test_solution" , 
                           "validate_data" e "validate_solution"

    

In [72]:
# 1. e 2. creato il file dataset.xlsx che contiene nello sheet "dataset" i 500 punti

# 3.
import numpy as np
import pandas as pd
data_xls = pd.read_excel('dataset.xlsx', 'dataset_simple', index_col=None) # pandas legge il file excel
data_xls.to_csv('dataset.csv', encoding='utf-8',index=False)  # salva il file in .csv

# NOTA : se index=True viene automaticamente aggiunta una colonna (la prima) contenente 
#        un indice progressivo crescente


In [73]:
# visualizziamo i primi 20 punti

# Reading the csv file into a pandas DataFrame
data_glob = pd.read_csv('dataset.csv')  # creato il DataFrame "data_glob"

# Printing out the first 10 rows of our data
data_glob[:10]


,x1,y,z
0,57,285,1
1,6,30,1
2,15,75,1
3,75,375,1
4,76,380,1
5,7,21,0
6,83,415,1
7,5,25,1
8,8,40,1
9,54,270,1


In [74]:
mean_=np.array([])
std_=np.array([])
for s in ['x1','y']:
    mean,std=np.array([data_glob[s].mean()]),np.array([data_glob[s].std()])
    data_glob.loc[:,s]=(data_glob[s]-mean)/std
    mean_=np.append([mean_],[mean])
    std_=np.append([std_],[std])
print (mean_,std_)

[ 51.444 207.464] [ 28.94405605 130.4284012 ]


In [75]:
# Printing out the first 10 rows of our data
data_glob[:10]

,x1,y,z
0,0.191957,0.594472,1
1,-1.570063,-1.360624,1
2,-1.259119,-1.015607,1
3,0.813846,1.284506,1
4,0.848395,1.322841,1
5,-1.535514,-1.429627,0
6,1.090241,1.591187,1
7,-1.604613,-1.398959,1
8,-1.500964,-1.283953,1
9,0.088308,0.479466,1


In [76]:
# 4. 

rows=len(data_glob.index)
columns=len(data_glob.columns)

train_part=0.9
test_part=0.5


from sklearn.model_selection import train_test_split

df_train,df_remain=train_test_split(data_glob,test_size=1-train_part,random_state=True)
df_test,df_validate=train_test_split(df_remain,test_size=1-test_part,random_state=True)

print ("The number of datas in the train set: ",df_train.size/columns)
print ("The number of datas in the test set: ",df_test.size/columns)
print ("The number of datas in the validation set: ",df_validate.size/columns)

The number of datas in the train set:  450.0
The number of datas in the test set:  25.0
The number of datas in the validation set:  25.0


In [77]:
# 4. bis  
input_features=columns-1

X_train=df_train.values[0:,0:input_features]
Y_train=df_train.values[:,input_features]

X_test=df_test.values[:,0:input_features]
Y_test=df_test.values[:,input_features]

X_validate=df_validate.values[:,0:input_features]
Y_validate=df_validate.values[:,input_features]

print("created data and solution for train set")
print("created data and solution for test set")
print("created data and solution for validate set")


created data and solution for train set
created data and solution for test set
created data and solution for validate set


   ## definiamo le funzioni base

In [141]:
# la rete avrà :
''' 1 layer
    n input (pari al numero di variabili indipendenti)
    m output (sempre 2 per la rete in esempio )'''
# date le dimensioni volute crea e inizializza una matrice dei pesi con media zero
# e deviazione standard 1\sqrt{n*m}

def weight_create(n,m):
    size=n*m
    weights = np.random.normal(loc=0.0,scale=1 / size**.5, size=(n,m))
    
    return (weights)

In [142]:
# create the matrix of Weights 

weights=weight_create(input_features,1)
print (weights)

[[-1.23585597]
 [ 0.65679923]]


In [143]:
def sigmoid(x):
    h=1/(1+np.exp(-x))
    return h

In [169]:
def sigmoid_prime(x):
    h=sigmoid(x)*(1+sigmoid(x))
    return h

In [188]:
def train(input_data,true_output,weights):
    
    epochs= 10000 # definiamo il numero di epoche
    learn_rate=0.5 # definiamo la learning rate
    #del_w = np.zeros(weights.shape)
    for e in range(epochs):
        
        #print (del_w)
        error_tot=0
        del_w = np.zeros(weights.shape)   # inizializzo a una matrice di zeri la variazione alla matrice weights
        #print (del_w) 
        #print ("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
        
        
        # poiche ogni epoca ho finito il calcolo di quanto deve variare 
        for x,y in zip(input_data,true_output):
            #print (x,y)
            #print (y.dtype)
            h=np.dot(x,weights)
            #print (h)
            y_hat=sigmoid(h)
            #print (h)
            error_i=(y-y_hat)
            
            error_term=error_i*sigmoid_prime(h)
            del_w=del_w+error_term*x[:,None]
            error_tot=error_tot+error_i
            
        #print (del_w)
        weights=weights+learn_rate*error_term#/(df_train.size/columns)
        error_tot=error_tot/(df_train.size/columns)
        print ("ERRORE OGNI EPOCH",error_tot)
        
        #print (weights)
        if e % (epochs / 5) == 0:
            out = sigmoid(np.dot(input_data, weights))
        
            loss = np.mean((out - true_output) ** 2)
            
            print ("aaaaaaa",loss)
            #print (del_w)
            
        
        #print (weights_)
        #print (Error)
    return (weights)
            
    

In [189]:
weights=train(X_train,Y_train,weights)
print (weights)

ERRORE OGNI EPOCH [0.46480868]
aaaaaaa 0.4975638060645008
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI 

ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE OGNI EPOCH [0.46480868]
ERRORE O

ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE OGNI EPOCH [0.46480869]
ERRORE O

KeyboardInterrupt: 

In [186]:
def predict(x1,y):
    weights_trained=weights
    x=np.array([x1,y])
    #print (mean_)
    x_=(x-mean_)/std_
    #print (x_)
    h=np.dot(x,weights_trained)
    #print (h)
    y_hat=sigmoid(h)
    return (y_hat)

In [187]:
#print(X_train.dtype)
#print (mean_)
print (predict(13,39)) # risultato 0
print (predict(13,65)) # risultato 1 
print (weights)

[1.]
[1.]
[[60.29299051]
 [62.18564571]]
